In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [2]:
# 定义超参数
input_size = 32  # 每行的输入大小
hidden_size = 256  # RNN隐藏状态的大小
num_classes = 10  # MNIST数据集的类别数
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [3]:
# 定义灰度化转换器
class GrayscaleTransform(object):
    def __call__(self, sample):
        gray_image = transforms.functional.rgb_to_grayscale(sample)
        return gray_image
# 加载CIFAR-10数据集并进行灰度化处理
transform = transforms.Compose([GrayscaleTransform(), transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='~/.cache', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [4]:
# 定义GRU模型
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

In [5]:
# 初始化模型和优化器
model = RNNClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# 训练模型
for epoch in range(num_epochs):
    total_loss = 0.0
    predictions = []
    ground_truth = []

    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        images = images.view(-1, 32, 32)  # 将每行作为输入
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        ground_truth.extend(labels.tolist())

    accuracy = accuracy_score(ground_truth, predictions)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss}, Accuracy: {accuracy:.2f}')

print('Training finished.')

Epoch 1/10: 100%|██████████| 782/782 [00:39<00:00, 19.61it/s]


Epoch [1/10], Loss: 1546.9602205753326, Accuracy: 0.26


Epoch 2/10: 100%|██████████| 782/782 [00:42<00:00, 18.20it/s]


Epoch [2/10], Loss: 1321.8384989500046, Accuracy: 0.39


Epoch 3/10: 100%|██████████| 782/782 [00:51<00:00, 15.32it/s]


...

Epoch [8/10], Loss: 902.6654099822044, Accuracy: 0.59


Epoch 9/10: 100%|██████████| 782/782 [00:37<00:00, 21.09it/s]


Epoch [9/10], Loss: 853.897329211235, Accuracy: 0.61


Epoch 10/10: 100%|██████████| 782/782 [01:34<00:00,  8.29it/s]

Epoch [10/10], Loss: 806.3722994327545, Accuracy: 0.64
Training finished.
